In [1]:
from __future__ import division
import numpy as np
import sys

In [2]:
k = 10
n = 200
d = 2

In [3]:
a = np.random.uniform(5, 10, [10, 2])
print a

[[ 6.59476825  9.47146056]
 [ 7.655011    9.5142869 ]
 [ 5.90806198  5.66544389]
 [ 7.5712928   6.14482785]
 [ 7.90994221  7.17580553]
 [ 5.13673768  5.02040451]
 [ 6.82766003  6.71988539]
 [ 5.25829271  7.34937448]
 [ 6.20171575  9.57980986]
 [ 6.39535643  7.54244146]]


In [4]:
ranges = 5*np.arange(11)

ranges2 = 2*np.arange(11)

x = np.array([]).astype(np.float32)
y = np.array([])

for i in range(k):
    for _ in range(10):
        x = np.append(x, np.random.uniform(ranges[i], ranges[i+1])).astype(np.float32)
        x = np.append(x, np.random.uniform(ranges2[i], ranges2[i+1])).astype(np.float32)
        
        y = np.append(y, i)

x = x.reshape(-1, d)

#print x
#print y

In [5]:
#X_train = np.genfromtxt(sys.argv[1], delimiter=",")
#y_train = np.genfromtxt(sys.argv[2])
#X_test = np.genfromtxt(sys.argv[3], delimiter=",")

X_train = x
y_train = y
X_test = x[np.random.choice(x.shape[0], 10, replace=False), :]

print X_test.shape
print y_train.size
print X_train[0].size

(10, 2)
100
2


In [6]:
def get_pi_hat(y_train):
    pi_hat = np.array([])

    for Y in range(k):
        indicator = 0

        for i in y_train:
            if i == Y:
                indicator += 1

        pi_hat = np.append(pi_hat, indicator/n)
        
    return pi_hat

In [7]:
def get_mu_hat(y_train, X_train):

    mu_hat = np.empty([0, d])

    for Y in range(k):

        indicator = 0
        mu_add = np.zeros((1, d))

        for i in range(y_train.size):
            if y_train[i] == Y:
                indicator += 1

                mu_add += X_train[i]

        mu_hat = np.append(mu_hat, mu_add/indicator, axis=0)
    
    return mu_hat

In [8]:
def get_sigma_hat(y_train, X_train, mu_hat):

    sigma_hat = np.empty([0, d, d])

    for Y in range(k):

        indicator = 0
        sigma_add = np.zeros((d, d))

        for i in range(y_train.size):
            if y_train[i] == Y:
                indicator += 1

                sigma_add += np.matmul((X_train[i]-mu_hat[Y]).reshape(2,1), np.transpose((X_train[i]-mu_hat[Y]).reshape(2,1)))

        sigma_hat = np.append(sigma_hat, sigma_add.reshape(1, d, d)/indicator, axis=0)
    
    return sigma_hat

In [9]:
i = 3

def classifier(X, pi_hat, mu_hat, sigma_hat):
    
    probabilities = np.array([])

    for Y in range(k):

        coefficient = pi_hat[Y]*np.linalg.norm(sigma_hat[Y])**(-0.5)

        exp = -0.5*np.matmul(np.matmul(np.transpose((X-mu_hat[Y]).reshape(d,1)),np.linalg.matrix_power(sigma_hat[Y],-1)), 
            (X-mu_hat[Y]).reshape(d,1))

        probability = coefficient*np.exp(exp)

        probabilities = np.append(probabilities, probability).astype(np.float32)
    
    return probabilities

#print classifier(X_test[i])
#print np.argmax(classifier(X_test[i]))

In [10]:
def pluginClassifier(X_train, y_train, X_test):    

    pi_hat = get_pi_hat(y_train)
    
    mu_hat = get_mu_hat(y_train, X_train)
    
    sigma_hat = get_sigma_hat(y_train, X_train, mu_hat)
    
    output = np.empty([0, k])
    
    for i in X_test:
        probabilities = classifier(i, pi_hat, mu_hat, sigma_hat)
        
        probabilities = (1/np.sum(probabilities))*probabilities
        
        output = np.append(output, probabilities.reshape(1,-1), axis=0)
    
    return output

pluginClassifier(X_train, y_train, X_test)

array([[  5.80924113e-23,   3.10783138e-17,   9.99997914e-01,
          2.10181065e-06,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   4.48565771e-08,
          9.99999821e-01,   3.67221134e-08,   1.69208279e-33,
          8.39960720e-38,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  9.99960065e-01,   3.99021010e-05,   1.26669342e-18,
          2.15799964e-43,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          2.27843907e-27,   1.29416556e-31,   3.16794671e-04,
          9.99683261e-01,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00

In [15]:
final_outputs = pluginClassifier(X_train, y_train, X_test)

print final_outputs.shape

np.savetxt("probs_test.csv", final_outputs, delimiter=",")

 (10, 10)


In [12]:
print np.sum(final_outputs[0])

for i in final_outputs:
    idk = 2*(np.exp(i)/(1+np.exp(i))-0.5)

    print idk, np.sum(idk)

1.00000001565
[  0.00000000e+00   0.00000000e+00   4.62116337e-01   1.05090533e-06
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00] 0.462117387836
[  0.00000000e+00   0.00000000e+00   2.24282888e-08   4.62117087e-01
   1.83610565e-08   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00] 0.462117127735
[  4.62101454e-01   1.99510505e-05   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00] 0.462121404726
[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   1.58397334e-04   4.61992599e-01   0.00000000e+00
   0.00000000e+00   0.00000000e+00] 0.462150996113
[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   2.04160084e-08   4.62117134e-01] 0.462117154238
[  0.00000000e+00   0.00000000e+00   0.00